<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Regression and Classification with the Ames Housing Data Project 



---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [3]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import category_encoders as ce
import patsy  

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [51]:
# Load the data
house = pd.read_csv('./housing.csv')

In [52]:
#removed all non-residential rows according to MSZoning values 
house = house[house['MSZoning'].str.contains('RL|RM|FV|RH', regex=True)]

In [53]:
house.MSZoning.unique()

array(['RL', 'RM', 'FV', 'RH'], dtype=object)

In [54]:
reno_house = house.filter(['OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtCond', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',\
                                   'HeatingQC', 'KitchenQual', 'BsmtQual', 'Functional', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence'])

In [55]:
#removed all mutable house characteristics 
#What I assumed was that the base features of the house would stay the same (such as no new house additions) and instead the base house would be improved. 
#Therefore, I removed all qualitative 'quality'-like variables and basement finish variables because finished/unfinished is like basement quality. 
fixed_house = house.drop(columns=['OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtCond', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',\
                                   'HeatingQC', 'KitchenQual', 'BsmtQual', 'Functional', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence'])

In [56]:
#dropped columns with many missing values 
#the Utilities column does not have no na values but the values are constant
fixed_house = fixed_house.drop(columns=['MiscFeature',
                                          'MiscVal', 'Utilities', 'Alley'])

In [57]:
fixed_house['TotalSF'] = fixed_house['1stFlrSF'] + fixed_house['2ndFlrSF']
fixed_house = fixed_house.drop(columns=['1stFlrSF', '2ndFlrSF'])

In [58]:
fixed_house['BsmtBath'] = fixed_house['BsmtFullBath'] \
+ (fixed_house['BsmtHalfBath'].apply(lambda x: x * 0.5))
fixed_house = fixed_house.drop(columns=\
                             ['BsmtFullBath', 'BsmtHalfBath'])

fixed_house['AbvGrBath'] = fixed_house['FullBath'] + \
(fixed_house['HalfBath'].apply(lambda x: x * 0.5))
fixed_house = fixed_house.drop(columns=['HalfBath', 'FullBath'])

In [59]:
#filling all null values with zero except Lot Frontage because
#unlike the others it doesn't seem plausable that all 229 are zero values 
values = {'MasVnrType': 0, 'MasVnrArea':0, 'BsmtExposure':0,\
          'GarageType':0, 'GarageYrBlt':0, 'GarageFinish':0, 'Electrical':1}

In [60]:
fixed_house = fixed_house.fillna(value=values)

In [61]:
#filling LotFrontage na values with the mean of the non-nulls 
fixed_house['LotFrontage'] = fixed_house['LotFrontage'].fillna(value=74.68)
fixed_house['GarageYrBlt'] = fixed_house['GarageYrBlt'].fillna(value=1978)

In [62]:
fixed_house.isna().sum()       

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
LotShape         0
LandContour      0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
Foundation       0
BsmtExposure     0
TotalBsmtSF      0
Heating          0
CentralAir       0
Electrical       0
LowQualFinSF     0
GrLivArea        0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
TotalSF          0
BsmtBath    

In [63]:
fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('RRNn', 'Transit')
fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('RRAn', 'Transit')
fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('RRNe', 'Transit')
fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('RRAe', 'Transit')

fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('Artery', 'MainRoad')
fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('Feedr', 'MainRoad')

fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('PosA', 'Greenspace')
fixed_house['Condition1'] = fixed_house['Condition1'].str.replace('PosN', 'Greenspace')

In [64]:
fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('RRNn', 'Transit')
fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('RRAn', 'Transit')
fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('RRNe', 'Transit')
fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('RRAe', 'Transit')

fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('Artery', 'MainRoad')
fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('Feedr', 'MainRoad')

fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('PosA', 'Greenspace')
fixed_house['Condition2'] = fixed_house['Condition2'].str.replace('PosN', 'Greenspace')

In [65]:
fixed_house['SaleCondition'] = fixed_house['SaleCondition'].str.replace('Alloca', 'Joint')
fixed_house['SaleCondition'] = fixed_house['SaleCondition'].str.replace('AdjLand', 'Joint')


In [66]:
fixed_house['SaleType'] = fixed_house['SaleType'].str.replace('ConLI', 'Con')
fixed_house['SaleType'] = fixed_house['SaleType'].str.replace('ConLw', 'Con')
fixed_house['SaleType'] = fixed_house['SaleType'].str.replace('ConLD', 'Con')
fixed_house['SaleType'] = fixed_house['SaleType'].str.replace('OthLw', 'Oth')
fixed_house['SaleType'] = fixed_house['SaleType'].str.replace('CWD', 'WD')

In [67]:
fixed_house['Foundation'] = fixed_house['Foundation'].str.replace('Slab', 'Oth')
fixed_house['Foundation'] = fixed_house['Foundation'].str.replace('Stone', 'Oth')
fixed_house['Foundation'] = fixed_house['Foundation'].str.replace('Wood', 'Oth')

In [68]:
fixed_house['MSSubClass'] = fixed_house['MSSubClass'].replace(40, 45)

In [69]:
fixed_house['LotConfig'] = fixed_house['LotConfig'].str.replace('FR2', 'Frontage')
fixed_house['LotConfig'] = fixed_house['LotConfig'].str.replace('FR3', 'Frontage')

In [70]:
fixed_house['BldgType'] = fixed_house['BldgType'].str.replace('2fmCon', 'Duplex')
fixed_house['BldgType'] = fixed_house['BldgType'].str.replace('TwnhsE', 'Twnhs')


In [71]:
fixed_house['RoofStyle'] = fixed_house['RoofStyle'].str.replace('Flat', 'Oth')
fixed_house['RoofStyle'] = fixed_house['RoofStyle'].str.replace('Gambrel', 'Oth')
fixed_house['RoofStyle'] = fixed_house['RoofStyle'].str.replace('Mansard', 'Oth')
fixed_house['RoofStyle'] = fixed_house['RoofStyle'].str.replace('Shed', 'Oth')

In [72]:
fixed_house['RoofMatl'] = fixed_house['RoofMatl'].str.replace('WdShngl', 'Oth')
fixed_house['RoofMatl'] = fixed_house['RoofMatl'].str.replace('WdShake', 'Oth')
fixed_house['RoofMatl'] = fixed_house['RoofMatl'].str.replace('Roll', 'Oth')
fixed_house['RoofMatl'] = fixed_house['RoofMatl'].str.replace('ClyTile', 'Oth')
fixed_house['RoofMatl'] = fixed_house['RoofMatl'].str.replace('Metal', 'Oth')
fixed_house['RoofMatl'] = fixed_house['RoofMatl'].str.replace('Membran', 'Oth')

In [73]:
fixed_house['Exterior1st'] = fixed_house['Exterior1st'].str.replace('BrkComm', 'Oth')
fixed_house['Exterior1st'] = fixed_house['Exterior1st'].str.replace('Stone', 'Oth')
fixed_house['Exterior1st'] = fixed_house['Exterior1st'].str.replace('ImStucc', 'Oth')
fixed_house['Exterior1st'] = fixed_house['Exterior1st'].str.replace('AsphShn', 'Oth')
fixed_house['Exterior1st'] = fixed_house['Exterior1st'].str.replace('CBlock', 'Oth')


In [74]:
fixed_house['Exterior2nd'] = fixed_house['Exterior2nd'].str.replace('Brk Cmn', 'Oth')
fixed_house['Exterior2nd'] = fixed_house['Exterior2nd'].str.replace('Stone', 'Oth')
fixed_house['Exterior2nd'] = fixed_house['Exterior2nd'].str.replace('ImStucc', 'Oth')
fixed_house['Exterior2nd'] = fixed_house['Exterior2nd'].str.replace('AsbShng', 'Oth')
fixed_house['Exterior2nd'] = fixed_house['Exterior2nd'].str.replace('CBlock', 'Oth')
fixed_house['Exterior2nd'] = fixed_house['Exterior2nd'].str.replace('Other', 'Oth')

In [75]:
fixed_house['MasVnrType'] = fixed_house['MasVnrType'].str.replace('0', 'None')

In [76]:
fixed_house['Heating'] = fixed_house['Heating'].str.replace('OthW', 'Oth')
fixed_house['Heating'] = fixed_house['Heating'].str.replace('Grav', 'Oth')
fixed_house['Heating'] = fixed_house['Heating'].str.replace('Wall', 'Oth')
fixed_house['Heating'] = fixed_house['Heating'].str.replace('Floor', 'Oth')

In [77]:
fixed_house['GarageType'] = fixed_house['GarageType'].str.replace('Basement', 'Attchd')
fixed_house['GarageType'] = fixed_house['GarageType'].str.replace('BuiltIn', 'Attchd')
fixed_house['GarageType'] = fixed_house['GarageType'].str.replace('0', 'None')
fixed_house['GarageType'] = fixed_house['GarageType'].str.replace('CarPort', 'Detchd')

In [78]:
fixed_house['GarageType'].value_counts()

Attchd     958
Detchd     389
Basment     18
2Types       6
Name: GarageType, dtype: int64

In [79]:
fixedOC = ce.ordinal.OrdinalEncoder([{'col': 'Street', 'mapping':[('Grvl', 3), ('Pave', 5)]},  
                                     {'col': 'LotShape', 'mapping':[('Reg', 5), ('IR1', 4), ('IR2', 3), ('IR3', 2)]},
                                     {'col': 'LandContour', 'mapping':[('Lvl', 5), ('Bnk', 3), ('HLS', 2), ('Low', 1)]}, 
                                     {'col': 'LandSlope', 'mapping': [('Gtl', 5), ('Mod', 3), ('Sev', 1)]},
                                     {'col': 'HouseStyle', 'mapping': [('1Story', 1), ('1.5Unf', 1.5), ('1.5Fin', 1.5), ('SFoyer', 1.5), ('SLvl', 2.0), ('2Story', 2), ('2.5Unf', 2.5), ('2.5Fin', 2.5)]},
                                     {'col': 'Electrical', 'mapping':[('SBrkr', 5), ('FuseA', 4), ('FuseF', 3),('Mix', 2), ('FuseP', 1), ('NA', 0)]},
                                     {'col': 'GarageFinish', 'mapping':[('Fin', 5), ('Rfn', 3), ('Unf', 1), ('NA', 0)]}], cols= ['Street', 'LotShape', 'LandContour', 'LandSlope', 'HouseStyle', 'Electrical', 'GarageFinish'])


In [80]:
fixed_house = fixedOC.fit_transform(fixed_house)

In [81]:
fixed_house.GarageFinish.value_counts()

2    600
1    419
3    352
4     79
Name: GarageFinish, dtype: int64

In [82]:
fixed_house.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street             int64
LotShape           int64
LandContour        int64
LotConfig         object
LandSlope          int64
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle         int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
Foundation        object
BsmtExposure      object
TotalBsmtSF        int64
Heating           object
CentralAir        object
Electrical         int64
LowQualFinSF       int64
GrLivArea          int64
BedroomAbvGr       int64
KitchenAbvGr       int64
TotRmsAbvGrd       int64
Fireplaces         int64
GarageType        object
GarageYrBlt      float64
GarageFinish       int64
GarageCars         int64
GarageArea         int64


In [83]:
categorical_fixed_house = fixed_house.filter(['MSZoning', 'LotConfig', 'LandSlope', 'Neighborhood',\
                                             'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', \
                                             'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'CentralAir', 'BsmtExposure', 'Heating', \
                                             'GarageType', 'SaleType', 'SaleCondition'])

In [84]:
print(categorical_fixed_house.shape)
print(pd.get_dummies(categorical_fixed_house).shape)

(1450, 20)
(1450, 106)


In [85]:
categorical_fixed_house = pd.get_dummies(categorical_fixed_house)

In [91]:
fixed_house.drop(columns=['MSZoning', 'LotConfig', 'LandSlope', 'Neighborhood',\
                                             'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', \
                                             'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'CentralAir', 'BsmtExposure', 'Heating', \
                                             'GarageType', 'SaleType', 'SaleCondition'], inplace=True)

KeyError: "['MSZoning' 'LotConfig' 'LandSlope' 'Neighborhood' 'Condition1'\n 'Condition2' 'BldgType' 'HouseStyle' 'RoofStyle' 'RoofMatl' 'Exterior1st'\n 'Exterior2nd' 'MasVnrType' 'Foundation' 'CentralAir' 'BsmtExposure'\n 'Heating' 'GarageType' 'SaleType' 'SaleCondition'] not found in axis"

In [92]:
fixed_house.shape

(1450, 34)

In [90]:
categorical_fixed_house.shape

(1450, 106)

In [93]:
dummies_fixed_house = pd.concat([fixed_house, categorical_fixed_house], axis=1)

In [94]:
dummies_fixed_house.shape

(1450, 140)

In [95]:
y_test = dummies_fixed_house[dummies_fixed_house['YrSold'] == 2010] 
y_test = np.array(y_test['SalePrice'])

y_train = dummies_fixed_house[dummies_fixed_house['YrSold'] < 2010]
y_train = np.array(y_train['SalePrice'])

x_test = dummies_fixed_house[dummies_fixed_house['YrSold'] == 2010]
x_test = x_test.drop(columns='SalePrice')

x_train = dummies_fixed_house[dummies_fixed_house['YrSold'] < 2010]
x_train = x_train.drop(columns='SalePrice')


In [42]:
x_train.isna().sum()

Id                       0
MSSubClass               0
LotFrontage              0
LotArea                  0
Street                   0
LotShape                 0
LandContour              0
YearBuilt                0
YearRemodAdd             0
MasVnrArea               0
TotalBsmtSF              0
Electrical               0
LowQualFinSF             0
GrLivArea                0
BedroomAbvGr             0
KitchenAbvGr             0
TotRmsAbvGrd             0
Fireplaces               0
GarageYrBlt              0
GarageFinish             0
GarageCars               0
GarageArea               0
WoodDeckSF               0
OpenPorchSF              0
EnclosedPorch            0
3SsnPorch                0
ScreenPorch              0
PoolArea                 0
MoSold                   0
YrSold                   0
                        ..
MasVnrType_None          0
MasVnrType_Stone         0
Foundation_BrkTil        0
Foundation_CBlock        0
Foundation_Oth           0
Foundation_PConc         0
C

In [96]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [97]:
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=x_test.columns)

In [98]:
#x_test = scaler.tranform(x_test)
#x_test = pd.DataFrame(x_test, columns=x_train.columns)

In [99]:
from sklearn.feature_selection import SelectKBest, \
mutual_info_regression, f_regression

In [49]:
mutual_reg = SelectKBest(mutual_info_regression, k=25)
mutual_reg.fit(x_train, y_train.ravel())

f_reg = SelectKBest(f_regression, k=25)
f_reg.fit(x_train, y_train.ravel())


SelectKBest(k=25, score_func=<function f_regression at 0x1d36412e60>)

In [101]:
kbest = pd.DataFrame([x_train.columns, mutual_reg.scores_, f_reg.scores_], 
                     index=['feature', 'mutual_reg', 'f_reg']).T.sort_values('mutual_reg', ascending=False)
kbest.head(10)

,feature,mutual_reg,f_reg
30,TotalSF,0.489548,1349.21
13,GrLivArea,0.475311,1284
7,YearBuilt,0.363424,453.593
10,TotalBsmtSF,0.360882,731.737
20,GarageCars,0.360479,894.937
21,GarageArea,0.359295,844.791
32,AbvGrBath,0.339502,716.619
1,MSSubClass,0.286939,7.02543
19,GarageFinish,0.264724,0.0694268
18,GarageYrBlt,0.248719,90.2281


In [104]:
post_feat_selection_x_train = x_train.iloc[:,[30,13,20,7,10,21,32,1,19,18]]
post_feat_selection_x_test = x_test.iloc[:,[30,13,20,7,10,21,32,1,19,18]]

In [105]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression() 

In [106]:
lm.fit(post_feat_selection_x_train, y_train)
lm.score(post_feat_selection_x_train, y_train)

0.7214999916430467

In [107]:
train_y_hat = lm.predict(post_feat_selection_x_test)

In [108]:
from sklearn.metrics import r2_score

In [112]:
from sklearn import metrics

In [114]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, train_y_hat))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, train_y_hat))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, train_y_hat)))

Mean Absolute Error: 104297547.91052634
Mean Squared Error: 1.1375249905642428e+16
Root Mean Squared Error: 106654816.60779521


In [314]:
sns.lmplot(x='')

133937.21753347918